## The steps of this code are as follows:
1. Import the RTKLIB corrected RINEX drone data
1. Import the Timestamp.MRK file created from the UAV
1. Create a calculation dataframe to correct imagery locations based on the corrected RTKLIB file
1. Create a new export file with updated Lat, Lon, and elevation and associated errors
1. Read the original exif data from the images and append that to your export file
1. Subtract the observed snow depth at the RTK site from each elevation observation
1. Export a final .csv with the updated lat, long, elevation and pitch, roll, yaw 

In [1]:
# Import numerical tools
import numpy as np
# Import pyplot for plotting
import matplotlib.pyplot as plt
#Import pandas for reading in and managing data
import pandas as pd
import math
# Magic function to make matplotlib inlineSet the filename for the code used for your imagery collection. This is the first 7 digits when you download imagery. Set the date for your flight collections.; other style specs must come AFTER
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}
#Comment the above line and uncomment the line below if svg graphics are not working in your browser.
#%config InlineBackend.figure_formats = {'png', 'retina'}
import os
import glob
import os.path

## Set the filename for the code used for your imagery collection. This is the first 7 digits when you download imagery. Make sure the date is in the same format as the folder that holds all your flight imagery on your computer. I store data in the form **'YYYYMMDD'**

In [68]:
# Set the date of the flight to search for the working directory for each time this code is run.
date = '20220124'

In [69]:
# Set your working directory
# Hourglass processing work should be in the folder 'HG_2022_PPK_Processing' and should be parsed out by date
# Since each date will have its own folder, the date in the working directory should be the only thing changed in
# this form.
os.chdir('/Users/f67f911/Desktop/HG_PPK/' + date)

## **Step 1:**
### Read in the already corrected RINEX file from the drone. This file should always be saved in your directory as filename_Rinex.csv so that the following code works. This is from the drone capturing positional location throughout the entirety of the flight, so there will be many more rows of data than there are pictures taken during that flight. 

In [70]:
# Designate the file paths for the raw and imagery files within the processing folder
raw_files_path = 'Raw_files/'
imagery_path = 'Imagery/'

In [71]:
# Capture all of the Rinex Files
rinex_files = glob.glob(raw_files_path + '/*.csv')
# And capture all of the timestamp files
timestamp_files = glob.glob(raw_files_path + '/*.MRK')

In [72]:
# Sort the files and check to make sure each has been populated correctly
# Sorting the data allows for easier processing later in the code
rinex_files = sorted(rinex_files)
rinex_files

['Raw_files/101_0250_Rinex.csv',
 'Raw_files/101_0251_Rinex.csv',
 'Raw_files/101_0252_Rinex.csv',
 'Raw_files/101_0253_Rinex.csv']

In [73]:
# Sort and check the timestamp files
timestamp_files = sorted(timestamp_files)
timestamp_files

['Raw_files/101_0250_Timestamp.MRK',
 'Raw_files/101_0251_Timestamp.MRK',
 'Raw_files/101_0252_Timestamp.MRK',
 'Raw_files/101_0253_Timestamp.MRK']

In [74]:
# Create an empty list to hold the data from each .csv file
rinex_dfholder = []
# For each file in the rinex files variable, read the csv file and append to the dfholder
for i in rinex_files:
        # Create a new dataframe for each file
        df = pd.read_csv(i, index_col = None, header = 0)
        df = pd.DataFrame(df)
        # Add flight number data to differentiate what flight each data was collected with
        flight = os.path.basename(i)
        flight = os.path.splitext(flight)[0]
        # Replace the 'Rinex' descriptor with blank characters
        flight = flight.replace('_Rinex', '')
        # Create a new column in the dataframe to hold the flight number values
        df['flight'] = flight
        # Append the list of df_holder with all of the values for the individual dataframes
        rinex_dfholder.append(df)
# Concat all of the data into a final dataframe
RTKLIB_record = pd.concat(rinex_dfholder, axis = 0, ignore_index = True)
RTKLIB_record

,%,GPST,latitude(deg),longitude(deg),height(m),Q,ns,sdn(m),sde(m),sdu(m),sdne(m),sdeu(m),sdun(m),age(s),ratio,flight
0,2194,151171.4,45.834875,-110.934339,2316.4640,1,5,0.0295,0.0316,0.0643,-0.0179,0.0201,-0.0240,-28.6,999.9,101_0250
1,2194,151171.6,45.834875,-110.934339,2316.4795,1,5,0.0293,0.0314,0.0639,-0.0178,0.0199,-0.0238,-28.4,999.9,101_0250
2,2194,151171.8,45.834875,-110.934338,2316.4897,1,5,0.0291,0.0312,0.0635,-0.0177,0.0198,-0.0237,-28.2,999.9,101_0250
3,2194,151172.0,45.834875,-110.934338,2316.4939,1,5,0.0289,0.0310,0.0631,-0.0175,0.0197,-0.0235,-28.0,999.9,101_0250
4,2194,151172.2,45.834876,-110.934338,2316.4884,1,5,0.0288,0.0308,0.0627,-0.0174,0.0196,-0.0234,-27.8,999.9,101_0250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8444,2194,156118.2,45.837361,-110.935108,2299.3347,1,6,0.0076,0.0077,0.0206,0.0049,0.0111,0.0064,28.2,999.9,101_0253
8445,2194,156118.4,45.837353,-110.935112,2299.3333,1,6,0.0076,0.0077,0.0204,0.0049,0.0109,0.0063,28.4,999.9,101_0253
8446,2194,156118.6,45.837345,-110.935115,2299.3394,1,6,0.0075,0.0073,0.0191,0.0047,0.0102,0.0059,28.6,999.9,101_0253
8447,2194,156118.8,45.837338,-110.935119,2299.3639,1,6,0.0072,0.0071,0.0186,0.0045,0.0099,0.0057,28.8,999.9,101_0253


## **Step 2:** 
### Read in the timestamp data as is from the UAV. 

In [75]:
# Create an empty list to hold the data from each .csv file
timestamp_dfholder = []
# For each file in the rinex files variable, read the csv file and append to the dfholder
for j in timestamp_files:
        # Create a new dataframe for each file
        df = pd.read_table(j, index_col = None, header = None)
        df = pd.DataFrame(df)
        # Add flight number data to differentiate what flight each data was collected with
        flight = os.path.basename(j)
        flight = os.path.splitext(flight)[0]
        # Replace the 'Rinex' descriptor with blank characters
        flight = flight.replace('_Timestamp', '')
        # Create a new column in the dataframe to hold the flight number values
        df['flight'] = flight
        timestamp_dfholder.append(df)
# Concat all of the data into a final dataframe
timestamp_record = pd.concat(timestamp_dfholder, axis = 0, ignore_index = True)
timestamp_record.columns = ['Photo', 'GPS_Date','% GPST','Northing_diff_mm','Easting_diff_mm','Elevation_diff_mm','Lat','Lon','Height_m','std_North_m, std_East_m, std_Ele_m','RTK_status_flag','flight']
timestamp_record.head()

,Photo,GPS_Date,% GPST,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Lat,Lon,Height_m,"std_North_m, std_East_m, std_Ele_m",RTK_status_flag,flight
0,1,151178.343990,[2194],"-22,N","-21,E","192,V","45.83487305,Lat","-110.93433503,Lon","2314.691,Ellh","0.017963, 0.017241, 0.036192","50,Q",101_0250
1,2,151183.143802,[2194],"-17,N","-15,E","193,V","45.83480542,Lat","-110.93446034,Lon","2321.249,Ellh","0.017807, 0.017057, 0.035862","50,Q",101_0250
2,3,151186.673368,[2194],"-14,N","-12,E","194,V","45.83474671,Lat","-110.93456976,Lon","2327.350,Ellh","0.017237, 0.016408, 0.032832","50,Q",101_0250
3,4,151190.230208,[2194],"-15,N","-5,E","194,V","45.83468599,Lat","-110.93467887,Lon","2333.419,Ellh","0.016899, 0.016118, 0.032532","50,Q",101_0250
4,5,151193.752063,[2194],"-12,N","-4,E","194,V","45.83462714,Lat","-110.93479034,Lon","2339.421,Ellh","0.017402, 0.016545, 0.033525","50,Q",101_0250


Note status flag values - 0: no positioning; 16: single-point positioning mode; 34: RTK floating solution; 50: RTK fixed solution. When flag of a photo is not equal to 50, it is recommended that you should not use that image in further processing.

## Clean the timestamp file to convert non-numeric text in columns to numeric

---
### When looking at our columns, we can see that there are many numbers followed by letters. We need to get rid of those numbers in order to continue with our analysis. 
### The following code removes any non-numeric value from the columns in the dataframe.

In [76]:
# Set the timestamp data to the timestamp record
timestamp = timestamp_record

In [77]:
# For each column in the timestamp dataframe
for col in timestamp:
    if col == 'flight':
        timestamp[col] = timestamp[col]
    # If the column name is '% GPST'
    elif col == '% GPST':
        # Remove the brackets around the value and change the data to type int
        timestamp[col] = timestamp[col].str.replace('[','', regex = True).str.replace(']','', regex = True).astype(int)
    # Or if the column is equal to the standard deviation column
    elif col == 'std_North_m, std_East_m, std_Ele_m':
        # Leave it alone
        timestamp[col] = timestamp[col]
    # If the data type of the column is an object
    elif timestamp.dtypes[col] == object:
        # Replace non-numeric values with a blank value
        timestamp[col] = timestamp[col].str.replace(r'\,\D+', '', regex = True).astype(float)
# View the data to make sure it has been changed correctly
timestamp.head()

,Photo,GPS_Date,% GPST,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Lat,Lon,Height_m,"std_North_m, std_East_m, std_Ele_m",RTK_status_flag,flight
0,1,151178.343990,2194,-22.0,-21.0,192.0,45.834873,-110.934335,2314.691,"0.017963, 0.017241, 0.036192",50.0,101_0250
1,2,151183.143802,2194,-17.0,-15.0,193.0,45.834805,-110.934460,2321.249,"0.017807, 0.017057, 0.035862",50.0,101_0250
2,3,151186.673368,2194,-14.0,-12.0,194.0,45.834747,-110.934570,2327.350,"0.017237, 0.016408, 0.032832",50.0,101_0250
3,4,151190.230208,2194,-15.0,-5.0,194.0,45.834686,-110.934679,2333.419,"0.016899, 0.016118, 0.032532",50.0,101_0250
4,5,151193.752063,2194,-12.0,-4.0,194.0,45.834627,-110.934790,2339.421,"0.017402, 0.016545, 0.033525",50.0,101_0250


### Calculate camera specific positions. 
Step one is to create a calculations spreadsheet.

In [78]:
# Create a new dataframe with set columns for our future calculations
calc = pd.DataFrame(columns = ['Northing_diff_mm','Easting_diff_mm','Elevation_diff_mm','Closest_Loc_ID',
                    'Timestamp_of_Closest','Closest_Lat','Closest_Lon','Closest_El','2nd_Closest_Loc_ID',
                    'Timestamp_of_2nd_Closest','2nd_Closest_Lat','2nd_Closest_Lon','2nd_Closest_El',
                    'Percent_diff_between_timestamps','Interpolated_Lat','Interpolated_Lon','Interpolated_El',
                    'Lat_Diff_deg','Lon_Diff_deg','El_diff_m','New_Lat','New_Lon','New_El']).astype(int)

### Calculate the values and input into the calc dataframe. This output will include a new latitude, longitude, and elevation.

In [79]:
# Populate the new dataframe with values from the timestamp data
calc['Northing_diff_mm'] = timestamp['Northing_diff_mm']
calc['Easting_diff_mm'] = timestamp['Easting_diff_mm']
calc['Elevation_diff_mm'] = timestamp['Elevation_diff_mm']
# Read in the data to make sure these columns have populated
calc.head()

,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Closest_Loc_ID,Timestamp_of_Closest,Closest_Lat,Closest_Lon,Closest_El,2nd_Closest_Loc_ID,Timestamp_of_2nd_Closest,...,Percent_diff_between_timestamps,Interpolated_Lat,Interpolated_Lon,Interpolated_El,Lat_Diff_deg,Lon_Diff_deg,El_diff_m,New_Lat,New_Lon,New_El
0,-22.0,-21.0,192.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-17.0,-15.0,193.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-14.0,-12.0,194.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-15.0,-5.0,194.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-12.0,-4.0,194.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## **Step 3:**
### Convert the latitude and longitude difference into degrees.

In [80]:
# First lets add a column in the RTKLIB data to populate out the timestamps.
# Set a record in case we mess up this data
RTKLIB = RTKLIB_record
RTKLIB.index.name = 'ID'
RTKLIB.reset_index(inplace = True)
RTKLIB.head()

,ID,%,GPST,latitude(deg),longitude(deg),height(m),Q,ns,sdn(m),sde(m),sdu(m),sdne(m),sdeu(m),sdun(m),age(s),ratio,flight
0,0,2194,151171.4,45.834875,-110.934339,2316.4640,1,5,0.0295,0.0316,0.0643,-0.0179,0.0201,-0.0240,-28.6,999.9,101_0250
1,1,2194,151171.6,45.834875,-110.934339,2316.4795,1,5,0.0293,0.0314,0.0639,-0.0178,0.0199,-0.0238,-28.4,999.9,101_0250
2,2,2194,151171.8,45.834875,-110.934338,2316.4897,1,5,0.0291,0.0312,0.0635,-0.0177,0.0198,-0.0237,-28.2,999.9,101_0250
3,3,2194,151172.0,45.834875,-110.934338,2316.4939,1,5,0.0289,0.0310,0.0631,-0.0175,0.0197,-0.0235,-28.0,999.9,101_0250
4,4,2194,151172.2,45.834876,-110.934338,2316.4884,1,5,0.0288,0.0308,0.0627,-0.0174,0.0196,-0.0234,-27.8,999.9,101_0250


In [81]:
# First, we need to create constant values with numbers used in further calculations
# as conversion factors
# 1 degree latitude in meters
deg_lat_m = 111111
# The latitude used. Use the first latitude value from the RTKLIB values
Lon_used = RTKLIB.iloc[0,RTKLIB.columns.get_loc('longitude(deg)')]
# 1 degree longitude in meters
deg_lon_m = abs(math.cos(np.deg2rad(Lon_used))*111111)

In [82]:
# This code calculates the latitude difference in degrees for this dataset. 
calc['Lat_Diff_deg'] = calc['Northing_diff_mm']/1000/deg_lat_m
# The longitude difference in degrees
calc['Lon_Diff_deg'] = calc['Easting_diff_mm']/1000/deg_lon_m
# The elevation difference in meters
calc['El_diff_m'] = calc['Elevation_diff_mm']/1000
# Call the head of the dataframe to make sure these calculations were done correctly
calc.head()

,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Closest_Loc_ID,Timestamp_of_Closest,Closest_Lat,Closest_Lon,Closest_El,2nd_Closest_Loc_ID,Timestamp_of_2nd_Closest,...,Percent_diff_between_timestamps,Interpolated_Lat,Interpolated_Lon,Interpolated_El,Lat_Diff_deg,Lon_Diff_deg,El_diff_m,New_Lat,New_Lon,New_El
0,-22.0,-21.0,192.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.980002e-07,-5.289710e-07,0.192,NaN,NaN,NaN
1,-17.0,-15.0,193.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.530002e-07,-3.778364e-07,0.193,NaN,NaN,NaN
2,-14.0,-12.0,194.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.260001e-07,-3.022691e-07,0.194,NaN,NaN,NaN
3,-15.0,-5.0,194.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.350001e-07,-1.259455e-07,0.194,NaN,NaN,NaN
4,-12.0,-4.0,194.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.080001e-07,-1.007564e-07,0.194,NaN,NaN,NaN


### Calculate the closest latitude. To do this, we first need to calculate the timestamp that is closest.

In [83]:
# Create a merged dataframe that merges the two dataframes on the closest time values
merge = pd.merge_asof(timestamp, RTKLIB.sort_values('GPST'), left_on = 'GPS_Date', right_on = 'GPST', 
                      direction = 'nearest')
# View the new column names to be able to populate the calc dataframe
merge.columns

Index(['Photo', 'GPS_Date', '% GPST', 'Northing_diff_mm', 'Easting_diff_mm',
       'Elevation_diff_mm', 'Lat', 'Lon', 'Height_m',
       'std_North_m, std_East_m, std_Ele_m', 'RTK_status_flag', 'flight_x',
       'ID', '%', 'GPST', 'latitude(deg)', 'longitude(deg)', 'height(m)', 'Q',
       'ns', 'sdn(m)', 'sde(m)', 'sdu(m)', 'sdne(m)', 'sdeu(m)', 'sdun(m)',
       'age(s)', 'ratio', 'flight_y'],
      dtype='object')

In [84]:
# Populate columns of the calc dataframe with data from the merged dataframe
calc['Closest_Loc_ID'] = merge['ID']
calc['Timestamp_of_Closest'] = merge['GPST']
calc['Closest_Lat'] = merge['latitude(deg)']
calc['Closest_Lon'] = merge['longitude(deg)']
calc['Closest_El'] = merge['height(m)']
# Add in the flight identifier column 
calc['flight'] = merge['flight_y']
# View to make sure it has been read in correctly
calc.head()

,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Closest_Loc_ID,Timestamp_of_Closest,Closest_Lat,Closest_Lon,Closest_El,2nd_Closest_Loc_ID,Timestamp_of_2nd_Closest,...,Interpolated_Lat,Interpolated_Lon,Interpolated_El,Lat_Diff_deg,Lon_Diff_deg,El_diff_m,New_Lat,New_Lon,New_El,flight
0,-22.0,-21.0,192.0,35,151178.4,45.834875,-110.934336,2316.4847,NaN,NaN,...,NaN,NaN,NaN,-1.980002e-07,-5.289710e-07,0.192,NaN,NaN,NaN,101_0250
1,-17.0,-15.0,193.0,59,151183.2,45.834806,-110.934463,2323.1065,NaN,NaN,...,NaN,NaN,NaN,-1.530002e-07,-3.778364e-07,0.193,NaN,NaN,NaN,101_0250
2,-14.0,-12.0,194.0,76,151186.6,45.834750,-110.934568,2328.9825,NaN,NaN,...,NaN,NaN,NaN,-1.260001e-07,-3.022691e-07,0.194,NaN,NaN,NaN,101_0250
3,-15.0,-5.0,194.0,94,151190.2,45.834688,-110.934679,2335.1314,NaN,NaN,...,NaN,NaN,NaN,-1.350001e-07,-1.259455e-07,0.194,NaN,NaN,NaN,101_0250
4,-12.0,-4.0,194.0,112,151193.8,45.834628,-110.934793,2341.2500,NaN,NaN,...,NaN,NaN,NaN,-1.080001e-07,-1.007564e-07,0.194,NaN,NaN,NaN,101_0250


In [85]:
# Now, we should populate the second closest values
# Add one to the closest location ID
calc['2nd_Closest_Loc_ID'] = calc['Closest_Loc_ID'] + 1
# And now populate the data from the RTKLIB file that matches that ID value
calc['Timestamp_of_2nd_Closest'] =  calc['2nd_Closest_Loc_ID'].map(RTKLIB['GPST'])
calc['2nd_Closest_Lat'] = calc['2nd_Closest_Loc_ID'].map(RTKLIB['latitude(deg)'])
calc['2nd_Closest_Lon'] = calc['2nd_Closest_Loc_ID'].map(RTKLIB['longitude(deg)'])
calc['2nd_Closest_El'] = calc['2nd_Closest_Loc_ID'].map(RTKLIB['height(m)'])
# View the dataframe to make sure it was populated correctly
calc.head()                                                 

,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Closest_Loc_ID,Timestamp_of_Closest,Closest_Lat,Closest_Lon,Closest_El,2nd_Closest_Loc_ID,Timestamp_of_2nd_Closest,...,Interpolated_Lat,Interpolated_Lon,Interpolated_El,Lat_Diff_deg,Lon_Diff_deg,El_diff_m,New_Lat,New_Lon,New_El,flight
0,-22.0,-21.0,192.0,35,151178.4,45.834875,-110.934336,2316.4847,36,151178.6,...,NaN,NaN,NaN,-1.980002e-07,-5.289710e-07,0.192,NaN,NaN,NaN,101_0250
1,-17.0,-15.0,193.0,59,151183.2,45.834806,-110.934463,2323.1065,60,151183.4,...,NaN,NaN,NaN,-1.530002e-07,-3.778364e-07,0.193,NaN,NaN,NaN,101_0250
2,-14.0,-12.0,194.0,76,151186.6,45.834750,-110.934568,2328.9825,77,151186.8,...,NaN,NaN,NaN,-1.260001e-07,-3.022691e-07,0.194,NaN,NaN,NaN,101_0250
3,-15.0,-5.0,194.0,94,151190.2,45.834688,-110.934679,2335.1314,95,151190.4,...,NaN,NaN,NaN,-1.350001e-07,-1.259455e-07,0.194,NaN,NaN,NaN,101_0250
4,-12.0,-4.0,194.0,112,151193.8,45.834628,-110.934793,2341.2500,113,151194.0,...,NaN,NaN,NaN,-1.080001e-07,-1.007564e-07,0.194,NaN,NaN,NaN,101_0250


In [86]:
# Calculate the percent difference between the timestamp values
calc['Percent_diff_between_timestamps'] = (timestamp['GPS_Date'] - calc['Timestamp_of_Closest'])/(calc['Timestamp_of_2nd_Closest'] - calc['Timestamp_of_Closest'])

In [87]:
# Calculate the interpolated latitude, longitude, and elevation based on the differences in timestamp data
calc['Interpolated_Lat'] = ((calc.Closest_Lat)*(1-calc.Percent_diff_between_timestamps))+(calc['2nd_Closest_Lat'])*(calc.Percent_diff_between_timestamps)
calc['Interpolated_Lon'] = ((calc.Closest_Lon)*(1-calc.Percent_diff_between_timestamps))+(calc['2nd_Closest_Lon'])*(calc.Percent_diff_between_timestamps)
calc['Interpolated_El'] = ((calc.Closest_El)*(1-calc.Percent_diff_between_timestamps))+(calc['2nd_Closest_El'])*(calc.Percent_diff_between_timestamps)
# View to make sure it has been populated correctly
calc

,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Closest_Loc_ID,Timestamp_of_Closest,Closest_Lat,Closest_Lon,Closest_El,2nd_Closest_Loc_ID,Timestamp_of_2nd_Closest,...,Interpolated_Lat,Interpolated_Lon,Interpolated_El,Lat_Diff_deg,Lon_Diff_deg,El_diff_m,New_Lat,New_Lon,New_El,flight
0,-22.0,-21.0,192.0,35,151178.4,45.834875,-110.934336,2316.4847,36,151178.6,...,45.834875,-110.934336,2316.486016,-1.980002e-07,-5.289710e-07,0.192,NaN,NaN,NaN,101_0250
1,-17.0,-15.0,193.0,59,151183.2,45.834806,-110.934463,2323.1065,60,151183.4,...,45.834807,-110.934461,2323.010261,-1.530002e-07,-3.778364e-07,0.193,NaN,NaN,NaN,101_0250
2,-14.0,-12.0,194.0,76,151186.6,45.834750,-110.934568,2328.9825,77,151186.8,...,45.834749,-110.934571,2329.116287,-1.260001e-07,-3.022691e-07,0.194,NaN,NaN,NaN,101_0250
3,-15.0,-5.0,194.0,94,151190.2,45.834688,-110.934679,2335.1314,95,151190.4,...,45.834688,-110.934680,2335.181077,-1.350001e-07,-1.259455e-07,0.194,NaN,NaN,NaN,101_0250
4,-12.0,-4.0,194.0,112,151193.8,45.834628,-110.934793,2341.2500,113,151194.0,...,45.834629,-110.934791,2341.167309,-1.080001e-07,-1.007564e-07,0.194,NaN,NaN,NaN,101_0250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,10.0,-12.0,194.0,8350,156099.4,45.836876,-110.934727,2299.4235,8351,156099.6,...,45.836874,-110.934726,2299.422289,9.000009e-08,-3.022691e-07,0.194,NaN,NaN,NaN,101_0253
467,6.0,-14.0,194.0,8371,156103.6,45.837013,-110.934820,2299.3483,8372,156103.8,...,45.837016,-110.934822,2299.349392,5.400005e-08,-3.526473e-07,0.194,NaN,NaN,NaN,101_0253
468,10.0,-12.0,194.0,8393,156108.0,45.837157,-110.934915,2299.3358,8394,156108.2,...,45.837159,-110.934917,2299.334385,9.000009e-08,-3.022691e-07,0.194,NaN,NaN,NaN,101_0253
469,13.0,-17.0,193.0,8415,156112.4,45.837300,-110.935012,2299.3562,8416,156112.6,...,45.837300,-110.935012,2299.355579,1.170001e-07,-4.282146e-07,0.193,NaN,NaN,NaN,101_0253


In [88]:
# Calculate the new latitude, longitude, and elevation
calc['New_Lat'] = calc['Interpolated_Lat'] + calc['Lat_Diff_deg']
calc['New_Lon'] = calc['Interpolated_Lon'] + calc['Lon_Diff_deg']
calc['New_El'] = calc['Interpolated_El'] + calc['El_diff_m']
calc

,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Closest_Loc_ID,Timestamp_of_Closest,Closest_Lat,Closest_Lon,Closest_El,2nd_Closest_Loc_ID,Timestamp_of_2nd_Closest,...,Interpolated_Lat,Interpolated_Lon,Interpolated_El,Lat_Diff_deg,Lon_Diff_deg,El_diff_m,New_Lat,New_Lon,New_El,flight
0,-22.0,-21.0,192.0,35,151178.4,45.834875,-110.934336,2316.4847,36,151178.6,...,45.834875,-110.934336,2316.486016,-1.980002e-07,-5.289710e-07,0.192,45.834875,-110.934337,2316.678016,101_0250
1,-17.0,-15.0,193.0,59,151183.2,45.834806,-110.934463,2323.1065,60,151183.4,...,45.834807,-110.934461,2323.010261,-1.530002e-07,-3.778364e-07,0.193,45.834807,-110.934462,2323.203261,101_0250
2,-14.0,-12.0,194.0,76,151186.6,45.834750,-110.934568,2328.9825,77,151186.8,...,45.834749,-110.934571,2329.116287,-1.260001e-07,-3.022691e-07,0.194,45.834748,-110.934571,2329.310287,101_0250
3,-15.0,-5.0,194.0,94,151190.2,45.834688,-110.934679,2335.1314,95,151190.4,...,45.834688,-110.934680,2335.181077,-1.350001e-07,-1.259455e-07,0.194,45.834688,-110.934680,2335.375077,101_0250
4,-12.0,-4.0,194.0,112,151193.8,45.834628,-110.934793,2341.2500,113,151194.0,...,45.834629,-110.934791,2341.167309,-1.080001e-07,-1.007564e-07,0.194,45.834629,-110.934791,2341.361309,101_0250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,10.0,-12.0,194.0,8350,156099.4,45.836876,-110.934727,2299.4235,8351,156099.6,...,45.836874,-110.934726,2299.422289,9.000009e-08,-3.022691e-07,0.194,45.836874,-110.934726,2299.616289,101_0253
467,6.0,-14.0,194.0,8371,156103.6,45.837013,-110.934820,2299.3483,8372,156103.8,...,45.837016,-110.934822,2299.349392,5.400005e-08,-3.526473e-07,0.194,45.837016,-110.934822,2299.543392,101_0253
468,10.0,-12.0,194.0,8393,156108.0,45.837157,-110.934915,2299.3358,8394,156108.2,...,45.837159,-110.934917,2299.334385,9.000009e-08,-3.022691e-07,0.194,45.837159,-110.934917,2299.528385,101_0253
469,13.0,-17.0,193.0,8415,156112.4,45.837300,-110.935012,2299.3562,8416,156112.6,...,45.837300,-110.935012,2299.355579,1.170001e-07,-4.282146e-07,0.193,45.837300,-110.935013,2299.548579,101_0253


## **Step 4:**
### Create a new dataframe for export that can be read into Agisoft

In [89]:
# Create an export dataframe that contains the new locational data
location_export = calc[['New_Lat','New_Lon','New_El','Northing_diff_mm','Easting_diff_mm','Elevation_diff_mm']]
# Reset the index to create a column to hold image ID data
location_export = location_export.reset_index()
# Rename the index column with a descriptive name and then incremently increase the values to match the collected imagery
location_export = location_export.rename(columns = {'index':'FlightID'})
# location_export['PhotoID'] = RTKLIB['flight']
location_export['FlightID'] = calc['flight'].astype(int)
location_export['Lat_diff_m'] = location_export['Northing_diff_mm']/1000
location_export['Lon_diff_m'] = location_export['Easting_diff_mm']/1000
location_export['El_diff_m'] = location_export['Elevation_diff_mm']/1000
location_export = location_export.drop(columns = ['Northing_diff_mm','Easting_diff_mm','Elevation_diff_mm'])
# View the data to make sure that it has been read in correctly.                                          
location_export

,FlightID,New_Lat,New_Lon,New_El,Lat_diff_m,Lon_diff_m,El_diff_m
0,1010250,45.834875,-110.934337,2316.678016,-0.022,-0.021,0.192
1,1010250,45.834807,-110.934462,2323.203261,-0.017,-0.015,0.193
2,1010250,45.834748,-110.934571,2329.310287,-0.014,-0.012,0.194
3,1010250,45.834688,-110.934680,2335.375077,-0.015,-0.005,0.194
4,1010250,45.834629,-110.934791,2341.361309,-0.012,-0.004,0.194
...,...,...,...,...,...,...,...
466,1010253,45.836874,-110.934726,2299.616289,0.010,-0.012,0.194
467,1010253,45.837016,-110.934822,2299.543392,0.006,-0.014,0.194
468,1010253,45.837159,-110.934917,2299.528385,0.010,-0.012,0.194
469,1010253,45.837300,-110.935013,2299.548579,0.013,-0.017,0.193


## **Step 5:**
### Now, we need to call the original photo EXIF information to adjust pitch, roll, and yaw. 

In [90]:
# Import exiftoolhelper from the package exiftool
from exiftool import ExifToolHelper

In [91]:
# Get the unique value for the flights and add the folder name to a list
imagery_folders = RTKLIB['flight'].unique().tolist()
imagery_folders

['101_0250', '101_0251', '101_0252', '101_0253']

In [92]:
# Create an empty list to hold the imagery names
imagery_files = []
# for each path in the imagery folders of interest, read in the .jpgs
for path in imagery_folders:
    # Append the empty list to create a list of lists
    imagery_files.append(glob.glob(imagery_path + path + '/*.JPG'))

In [93]:
# Create a loop to turn the list of lists generated in the above code to a flat list
imagery_flat_list = []
# For each sublist in the above list of lists, append the item of that list to a singular list
for sublist in imagery_files:
    for item in sublist:
        imagery_flat_list.append(item)

In [94]:
# Create an empty list to hold the exif_data
exif_data = []
# With the ExifToolHelper, get tag information for each image
with ExifToolHelper() as et:
    # Specify what tag information you would like to capture
    for d in et.get_tags(imagery_flat_list, tags = ['Pitch','Roll','Yaw']):
        # append the list 
        exif_data.append(d)
        # Create a dataframe from the list values 
        exif_df = pd.DataFrame.from_dict(exif_data)

In [95]:
exif_df = exif_df.sort_values(by = 'SourceFile')
exif_df = exif_df.reset_index(drop = True)
exif_df

,SourceFile,MakerNotes:Pitch,MakerNotes:Roll,MakerNotes:Yaw
0,Imagery/101_0250/101_0250_0001.JPG,-1.6,1.900000,-126.199997
1,Imagery/101_0250/101_0250_0002.JPG,-3.6,1.600000,-127.300003
2,Imagery/101_0250/101_0250_0003.JPG,-4.9,1.200000,-127.800003
3,Imagery/101_0250/101_0250_0004.JPG,-6.2,2.400000,-127.000000
4,Imagery/101_0250/101_0250_0005.JPG,-7.2,2.200000,-128.000000
...,...,...,...,...
466,Imagery/101_0253/101_0253_0063.JPG,-5.8,2.400000,-25.000000
467,Imagery/101_0253/101_0253_0064.JPG,-7.1,3.100000,-24.400000
468,Imagery/101_0253/101_0253_0065.JPG,-5.8,1.900000,-25.400000
469,Imagery/101_0253/101_0253_0066.JPG,-4.6,3.300000,-24.100000


In [96]:
file_names = []
for f in exif_df['SourceFile']:
    name = os.path.basename(f)
    file_names.append(name)
exif_df['file_names'] = file_names
exif_df

,SourceFile,MakerNotes:Pitch,MakerNotes:Roll,MakerNotes:Yaw,file_names
0,Imagery/101_0250/101_0250_0001.JPG,-1.6,1.900000,-126.199997,101_0250_0001.JPG
1,Imagery/101_0250/101_0250_0002.JPG,-3.6,1.600000,-127.300003,101_0250_0002.JPG
2,Imagery/101_0250/101_0250_0003.JPG,-4.9,1.200000,-127.800003,101_0250_0003.JPG
3,Imagery/101_0250/101_0250_0004.JPG,-6.2,2.400000,-127.000000,101_0250_0004.JPG
4,Imagery/101_0250/101_0250_0005.JPG,-7.2,2.200000,-128.000000,101_0250_0005.JPG
...,...,...,...,...,...
466,Imagery/101_0253/101_0253_0063.JPG,-5.8,2.400000,-25.000000,101_0253_0063.JPG
467,Imagery/101_0253/101_0253_0064.JPG,-7.1,3.100000,-24.400000,101_0253_0064.JPG
468,Imagery/101_0253/101_0253_0065.JPG,-5.8,1.900000,-25.400000,101_0253_0065.JPG
469,Imagery/101_0253/101_0253_0066.JPG,-4.6,3.300000,-24.100000,101_0253_0066.JPG


In [97]:
final_export = pd.merge(location_export, exif_df, left_index = True, right_index = True)
final_export

,FlightID,New_Lat,New_Lon,New_El,Lat_diff_m,Lon_diff_m,El_diff_m,SourceFile,MakerNotes:Pitch,MakerNotes:Roll,MakerNotes:Yaw,file_names
0,1010250,45.834875,-110.934337,2316.678016,-0.022,-0.021,0.192,Imagery/101_0250/101_0250_0001.JPG,-1.6,1.900000,-126.199997,101_0250_0001.JPG
1,1010250,45.834807,-110.934462,2323.203261,-0.017,-0.015,0.193,Imagery/101_0250/101_0250_0002.JPG,-3.6,1.600000,-127.300003,101_0250_0002.JPG
2,1010250,45.834748,-110.934571,2329.310287,-0.014,-0.012,0.194,Imagery/101_0250/101_0250_0003.JPG,-4.9,1.200000,-127.800003,101_0250_0003.JPG
3,1010250,45.834688,-110.934680,2335.375077,-0.015,-0.005,0.194,Imagery/101_0250/101_0250_0004.JPG,-6.2,2.400000,-127.000000,101_0250_0004.JPG
4,1010250,45.834629,-110.934791,2341.361309,-0.012,-0.004,0.194,Imagery/101_0250/101_0250_0005.JPG,-7.2,2.200000,-128.000000,101_0250_0005.JPG
...,...,...,...,...,...,...,...,...,...,...,...,...
466,1010253,45.836874,-110.934726,2299.616289,0.010,-0.012,0.194,Imagery/101_0253/101_0253_0063.JPG,-5.8,2.400000,-25.000000,101_0253_0063.JPG
467,1010253,45.837016,-110.934822,2299.543392,0.006,-0.014,0.194,Imagery/101_0253/101_0253_0064.JPG,-7.1,3.100000,-24.400000,101_0253_0064.JPG
468,1010253,45.837159,-110.934917,2299.528385,0.010,-0.012,0.194,Imagery/101_0253/101_0253_0065.JPG,-5.8,1.900000,-25.400000,101_0253_0065.JPG
469,1010253,45.837300,-110.935013,2299.548579,0.013,-0.017,0.193,Imagery/101_0253/101_0253_0066.JPG,-4.6,3.300000,-24.100000,101_0253_0066.JPG


In [98]:
final_export = final_export.drop(columns = ['SourceFile','FlightID'])
final_export = final_export.rename(columns = {'MakerNotes:Pitch':'Pitch','MakerNotes:Roll':'Roll','MakerNotes:Yaw':'Yaw'})
final_export = final_export.set_index('file_names')

In [99]:
final_export = final_export.sort_values(by = 'file_names')
final_export

,New_Lat,New_Lon,New_El,Lat_diff_m,Lon_diff_m,El_diff_m,Pitch,Roll,Yaw
file_names,,,,,,,,,
101_0250_0001.JPG,45.834875,-110.934337,2316.678016,-0.022,-0.021,0.192,-1.6,1.900000,-126.199997
101_0250_0002.JPG,45.834807,-110.934462,2323.203261,-0.017,-0.015,0.193,-3.6,1.600000,-127.300003
101_0250_0003.JPG,45.834748,-110.934571,2329.310287,-0.014,-0.012,0.194,-4.9,1.200000,-127.800003
101_0250_0004.JPG,45.834688,-110.934680,2335.375077,-0.015,-0.005,0.194,-6.2,2.400000,-127.000000
101_0250_0005.JPG,45.834629,-110.934791,2341.361309,-0.012,-0.004,0.194,-7.2,2.200000,-128.000000
...,...,...,...,...,...,...,...,...,...
101_0253_0063.JPG,45.836874,-110.934726,2299.616289,0.010,-0.012,0.194,-5.8,2.400000,-25.000000
101_0253_0064.JPG,45.837016,-110.934822,2299.543392,0.006,-0.014,0.194,-7.1,3.100000,-24.400000
101_0253_0065.JPG,45.837159,-110.934917,2299.528385,0.010,-0.012,0.194,-5.8,1.900000,-25.400000


## **Step 6:**
### Export the final cleaned dataframe to a .csv file to be used in Agisoft

In [100]:
#final_export.to_csv(date + 'Corrected_Imagery_Locs.csv')
final_export.to_csv(date + 'NoHeightChange.csv')